In [7]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [8]:
FEATURES_PATH = Path("../data/all_features/data")


In [20]:
filepaths = list(FEATURES_PATH.glob("*.jsonl"))
num_swaps = {}
for fp in tqdm(filepaths):
    df = pd.read_json(fp, lines=True)
    try:
        num_swaps[fp.name] = df["is_swapped"].value_counts()[True]
    except Exception:
        num_swaps[fp.name] = len(df)

100%|██████████| 60/60 [00:07<00:00,  8.31it/s]


In [46]:
swaps_df = pd.DataFrame([num_swaps]).transpose().rename(columns={0:"num_swaps"}).reset_index().rename(columns={"index": "filename"})

In [40]:
result_df = pd.read_csv("../data/Human Pref Data Model Approach - Sheet1.csv").rename(columns={"Unnamed: 0": "experiment_id"})


In [67]:
def get_id(name):
    _, feats = name.split("FEATS")
    feats = feats.removesuffix(".jsonl")
    return "rm-eval-tulu2_13b_rm_human_datamodel_7000_" + "FEATS" + feats

swaps_df["experiment_id"] = swaps_df["filename"].apply(lambda x: get_id(x))

In [72]:
combined = result_df.merge(swaps_df, how="left", on="experiment_id")

In [73]:
combined.to_csv("combined.csv")